# Electricity Price Prediction - Backfill Pipeline

## 🗒️ Overview
This notebook collects historical data for electricity price prediction in Stockholm (SE3):
1. **Electricity prices** from elprisetjustnu.se API (hourly data from Nov 2022)
2. **Weather data** from Open-Meteo API (hourly historical data)

The goal is to train a model that predicts electricity prices for each hour of the next day.

### Weather Features Selected
We use weather features that affect electricity supply and demand:
- **Temperature** → heating/cooling demand
- **Wind speed** (10m & 100m) → wind power generation
- **Cloud cover** → solar power generation
- **Precipitation** → hydro power availability

In [ ]:
from pathlib import Path
import sys
import pandas as pd
from datetime import date, timedelta
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import hopsworks

# 1. Find project root (one level up from notebooks/)
root_dir = Path("..").resolve()

# 2. Add project root to PYTHONPATH so we can import the src package
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# 3. Load .env from project root
env_path = root_dir / ".env"
load_dotenv(env_path)

# 4. Load settings and utility functions (after adjusting PYTHONPATH)
from src.config import ElectricitySettings
from src import util

settings = ElectricitySettings()

# 5. Log in to Hopsworks and get feature store
project = hopsworks.login(engine="python")
fs = project.get_feature_store()

print("Successfully logged in to Hopsworks project:", settings.HOPSWORKS_PROJECT)
print(f"Feature Store: {fs}")

# Show the weather variables we'll be using
print(f"\nWeather variables: {util.HOURLY_WEATHER_VARIABLES}")

## ⚙️ Configuration

Define the price area and date range for historical data collection.


In [ ]:
# Configuration
PRICE_AREA = "SE3"  # Stockholm / Södra Mellansverige
CITY = "Stockholm"
LATITUDE = 59.3251   # Stockholm coordinates
LONGITUDE = 18.0711

#LATITUDE, LONGITUDE = util.get_city_coordinates(CITY)

# Historical data range
# Electricity prices available from Nov 1, 2022
START_DATE = date(2022, 11, 1)
#START_DATE = date(2025, 12, 10)
END_DATE = date.today()  

print(f"Price Area: {PRICE_AREA}")
print(f"City: {CITY} ({LATITUDE}, {LONGITUDE})")
print(f"Date range: {START_DATE} to {END_DATE}")
print(f"Total days to fetch: {(END_DATE - START_DATE).days + 1}")


## ⚡ Step 1: Fetch Historical Electricity Prices

Using the elprisetjustnu.se API to get hourly electricity prices for Stockholm (SE3).


In [ ]:
# Using fetch_electricity_prices() from util.py
df_prices = util.fetch_electricity_prices(START_DATE, END_DATE, PRICE_AREA)
#df_prices = util.align_electricity_price_schema(df_prices)

# Ensure timezone-aware datetime and unix_time; keep only 'date'
df_prices['date'] = pd.to_datetime(df_prices['timestamp'], utc=True)
df_prices['unix_time'] = df_prices['date'].astype('int64') // 10**6
df_prices = df_prices.drop(columns=['timestamp'])

# Use price area label consistently
df_prices['price_area'] = PRICE_AREA.lower()



In [ ]:
df_prices['price_area'] = df_prices['price_area'].astype('string')
df_prices.head()

In [ ]:
# Check the electricity prices data
print(f"Shape: {df_prices.shape}")
print(f"\nDate range: {df_prices['date'].min()} to {df_prices['date'].max()}")
print(f"\nColumn types:")
df_prices.info()


## 🌦 Step 2: Fetch Historical Weather Data

Using Open-Meteo API to get hourly weather data that may correlate with electricity prices:
- Temperature affects heating/cooling demand
- Wind speed affects wind power generation
- Cloud cover affects solar power generation
- Precipitation can affect hydro power


In [ ]:
# Using get_hourly_historical_weather() from electricity_utils.py
# Fetch hourly weather data for the date range

df_weather = util.get_hourly_historical_weather(
    latitude=LATITUDE,
    longitude=LONGITUDE, 
    start_date=str(pd.to_datetime(df_prices['date'].min()).date()),
    end_date=str(END_DATE),
    city=PRICE_AREA.lower()
)

# Align label with price area naming
if 'city' in df_weather.columns:
    df_weather.rename(columns={'city': 'price_area'}, inplace=True)

df_weather.head()


In [ ]:
# Check the weather data
print(f"Shape: {df_weather.shape}")
print(f"\nDate range: {df_weather['date'].min()} to {df_weather['date'].max()}")
print(f"\nWeather features: {[c for c in df_weather.columns if c not in ['price_area', 'date', 'hour']]}")


In [ ]:
# Show weather statistics
print("\nTemperature statistics (°C):")
print(df_weather['temperature_2m'].describe())
print("\nWind speed at 100m (km/h):")
print(df_weather['wind_speed_100m'].describe())


## 🔧 Step 3: Data Processing

Clean and prepare the data for the feature store.


In [ ]:
# The utility functions already handle type conversions and cleaning
# Ensure unix_time exists for primary key
if 'unix_time' not in df_prices.columns:
    df_prices['unix_time'] = pd.to_datetime(df_prices['date'], utc=True).astype('int64') // 10**6

# Drop rows with missing values (align with Lab1 pattern)
df_prices = df_prices.dropna().reset_index(drop=True)

print("Electricity prices ready for feature store:")
print(f"  Shape: {df_prices.shape}")
print(f"  Columns: {list(df_prices.columns)}")
df_prices.info()


In [ ]:
# The utility functions already handle type conversions and cleaning
# Ensure timezone-aware datetime columns; keep only 'date'
df_weather['date'] = pd.to_datetime(df_weather['timestamp'], utc=True)
df_weather['unix_time'] = df_weather['date'].astype('int64') // 10**6
df_weather = df_weather.drop(columns=['timestamp'])
df_weather['price_area'] = df_weather['price_area'].astype('string')
if 'city' in df_weather.columns:
    df_weather = df_weather.drop(columns=['city'])

# Drop rows with missing values
df_weather = df_weather.dropna().reset_index(drop=True)

print("Weather data ready for feature store:")
print(f"  Shape: {df_weather.shape}")
print(f"  Columns: {list(df_weather.columns)}")
df_weather.info()


In [ ]:
# Check for missing values
print("Missing values in electricity prices:")
print(df_prices.isnull().sum())
print(f"\n{'='*50}\n")
print("Missing values in weather data:")
print(df_weather.isnull().sum())


## ✅ Step 4: Data Validation

Define validation rules using Great Expectations to ensure data quality.


In [ ]:
# Add unix_time (ms) for online FG primary key
if 'unix_time' not in df_weather.columns:
    df_weather['unix_time'] = pd.to_datetime(df_weather['date'], utc=True).astype('int64') // 10**6

print("Added unix_time to weather data:")
print(f"  Columns: {list(df_weather.columns)}")
print(df_weather[['date', 'unix_time']].head())


In [ ]:
import great_expectations as ge

# Expectation suite for electricity prices
price_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="electricity_price_expectations"
)

# Price should be reasonable (can be negative in some cases, but typically between -1 and 10 SEK/kWh)
price_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "price_sek",
            "min_value": -5.0,  # Prices can occasionally be negative
            "max_value": 50.0,   # Upper bound sanity check
            "strict_min": False
        }
    )
)

# Hour should be between 0 and 23
price_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "hour",
            "min_value": 0,
            "max_value": 23
        }
    )
)

print("Price expectation suite created")


In [ ]:
# Expectation suite for weather data
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectations"
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "temperature_2m",
            "min_value": -20.0,
            "max_value": 40.0
        }
    )
)

# Wind speed should be non-negative
weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "wind_speed_10m",
            "min_value": -0.1,
            "max_value": 200.0,  # Max reasonable wind speed
            "strict_min": False
        }
    )
)

# Precipitation should be non-negative
weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column": "precipitation",
            "min_value": -0.1,
            "max_value": 500.0,
            "strict_min": False
        }
    )
)

print("Weather expectation suite created")


## 💾 Step 5: Create Feature Groups in Hopsworks

Create feature groups for electricity prices and weather data, then insert the historical data.


In [ ]:
# Add unix_time (ms) for online FG primary key (normalized approach)
# ensure UTC and then convert to milliseconds
if 'unix_time' in df_weather.columns:
    df_weather = df_weather.drop(columns=['unix_time'])
df_weather['date'] = pd.to_datetime(df_weather['date'], utc=True)
df_weather['unix_time'] = df_weather['date'].astype('int64') // 10**6

print("Added unix_time to weather data (normalized):")
print(f"  Columns: {list(df_weather.columns)}")
print(df_weather[['date', 'unix_time']].head())


In [ ]:
# Create electricity prices feature group (online-only, SEK only)
electricity_fg = fs.get_or_create_feature_group(
    name='electricity_prices',
    description='Hourly electricity prices for Swedish price areas (SEK only)',
    version=1,                                  
    primary_key=['price_area', 'unix_time'],    
    event_time='date',                          
    expectation_suite=price_expectation_suite,
    online_enabled=True,
)

electricity_fg.insert(df_prices, wait=True)

print(f"Feature group created: {electricity_fg.name} v{electricity_fg.version}")

In [ ]:
# Add feature descriptions for electricity prices (SEK only)
electricity_fg.update_feature_description("unix_time", "Timestamp in unix epoch milliseconds (Primary Key)")
electricity_fg.update_feature_description("date", "Timestamp of the price period start (hourly)")
electricity_fg.update_feature_description("hour", "Hour of the day (0-23)")
electricity_fg.update_feature_description("price_area", "Swedish electricity price area (SE1-SE4)")
electricity_fg.update_feature_description("price_sek", "Electricity price in SEK per kWh (excl. VAT)")

print("Feature descriptions added for electricity prices")


In [ ]:
# Create weather feature group (online-only to avoid HopsFS)
weather_fg = fs.get_or_create_feature_group(
    name='weather_hourly',
    description='Hourly weather data for electricity price prediction',
    version=1,
    primary_key=['price_area', 'unix_time'],
    event_time='date',
    expectation_suite=weather_expectation_suite,
    online_enabled=True,
)

print(f"Feature group created: {weather_fg.name} v{weather_fg.version}")


In [ ]:
weather_fg.insert(df_weather, wait=True)


In [ ]:
# Add feature descriptions for weather data
# These match the variables defined in HOURLY_WEATHER_VARIABLES in util.py
weather_fg.update_feature_description("unix_time", "Timestamp in unix epoch milliseconds (Primary Key)")
weather_fg.update_feature_description("date", "Timestamp of the weather measurement (hourly)")
weather_fg.update_feature_description("hour", "Hour of the day (0-23)")
weather_fg.update_feature_description("price_area", "Swedish electricity price area (SE1-SE4)")

# Temperature features
weather_fg.update_feature_description("temperature_2m", "Air temperature at 2m height in °C")
weather_fg.update_feature_description("apparent_temperature", "Feels-like temperature in °C (affects heating/cooling demand)")

# Precipitation features
weather_fg.update_feature_description("precipitation", "Total precipitation (rain + snow) in mm")
weather_fg.update_feature_description("rain", "Rainfall in mm")
weather_fg.update_feature_description("snowfall", "Snowfall in cm")

# Cloud cover (affects solar power)
weather_fg.update_feature_description("cloud_cover", "Total cloud cover in % (affects solar power generation)")

# Wind features (affects wind power)
weather_fg.update_feature_description("wind_speed_10m", "Wind speed at 10m in km/h")
weather_fg.update_feature_description("wind_speed_100m", "Wind speed at 100m (turbine height) in km/h - key for wind power")
weather_fg.update_feature_description("wind_direction_10m", "Wind direction at 10m in degrees")
weather_fg.update_feature_description("wind_direction_100m", "Wind direction at 100m in degrees")
weather_fg.update_feature_description("wind_gusts_10m", "Wind gusts at 10m in km/h (can cause turbine shutdowns)")

# Pressure
weather_fg.update_feature_description("surface_pressure", "Surface pressure in hPa (weather patterns)")

print("Feature descriptions added for weather data")


## 🔐 Step 6: Save Configuration as Secrets

Store the location configuration in Hopsworks secrets for use in daily pipelines.


In [ ]:
import json

# Save location configuration as a Hopsworks secret
location_config = {
    "price_area": PRICE_AREA,
    "city": CITY,
    "latitude": LATITUDE,
    "longitude": LONGITUDE
}

location_str = json.dumps(location_config)

# Get secrets API
secrets = hopsworks.get_secrets_api()

# Save or update the location secret
secret_name = "ELECTRICITY_LOCATION_JSON"
try:
    existing_secret = secrets.get_secret(secret_name)
    if existing_secret is not None:
        existing_secret.delete()
        print(f"Replacing existing {secret_name}")
except:
    pass

secrets.create_secret(secret_name, location_str)
print(f"Saved location configuration to secret: {secret_name}")
print(f"Config: {location_config}")


## ✅ Summary

Backfill complete! We have created:

1. **electricity_prices** feature group with hourly prices from elprisetjustnu.se
2. **weather_hourly** feature group with hourly weather data from Open-Meteo

### Next Steps
- Create a **daily feature pipeline** to update with new data
- Create a **training pipeline** to build a prediction model
- Create a **batch inference pipeline** to generate daily predictions


In [ ]:
# Final summary
print("=" * 60)
print("BACKFILL COMPLETE")
print("=" * 60)
print(f"\n📊 Electricity Prices:")
print(f"   - Records: {len(df_prices):,}")
print(f"   - Date range: {df_prices['date'].min()} to {df_prices['date'].max()}")
print(f"   - Price area: {PRICE_AREA}")

print(f"\n🌦 Weather Data:")
print(f"   - Records: {len(df_weather):,}")
print(f"   - Date range: {df_weather['date'].min()} to {df_weather['date'].max()}")
print(f"   - Price area: {PRICE_AREA}")

print(f"\n🔗 Hopsworks Feature Groups:")
print(f"   - {electricity_fg.name} (v{electricity_fg.version})")
print(f"   - {weather_fg.name} (v{weather_fg.version})")
